In [60]:
import torch
import os
from pathlib import Path
import warnings
from tqdm import tqdm

from razdel import sentenize
import fitz
import re

import pandas as pd
import seaborn as sns

warnings.filterwarnings('ignore')

In [5]:
df = pd.read_csv("data/habr/raw_texts.csv")

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 166320 entries, 0 to 166319
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  166320 non-null  int64 
 1   id          166320 non-null  int64 
 2   title       166320 non-null  object
 3   text        165728 non-null  object
 4   tags        166320 non-null  object
dtypes: int64(2), object(3)
memory usage: 6.3+ MB


In [11]:
def custom_str_to_lst(string):
    return string[1:-1].replace("'", "").split(", ")

In [12]:
df['tags'] = df.tags.apply(custom_str_to_lst)

In [35]:
(df['tags']
 .explode()
 .apply(lambda x: x.lower())
 .value_counts()
 .index[:200]
 .tolist())

['google',
 'android',
 'javascript',
 'microsoft',
 'linux',
 'php',
 'apple',
 'java',
 'python',
 'программирование',
 'стартапы',
 'разработка',
 'ios',
 'стартап',
 'социальные сети',
 'iphone',
 '.net',
 'яндекс',
 'windows',
 'css',
 'игры',
 'дизайн',
 'c++',
 'open source',
 'хабрахабр',
 'безопасность',
 'интернет',
 '',
 'facebook',
 'реклама',
 'информационная безопасность',
 'видео',
 'юмор',
 'firefox',
 'конференция',
 'c#',
 'html',
 'html5',
 'ubuntu',
 'opera',
 'статистика',
 'юзабилити',
 'тестирование',
 'обзор',
 'маркетинг',
 'управление проектами',
 'перевод',
 'ruby',
 'jquery',
 'работа',
 'хостинг',
 'twitter',
 'бизнес',
 'nokia',
 'web-разработка',
 'конкурс',
 'api',
 'браузеры',
 'mail.ru',
 'mysql',
 'новости',
 'образование',
 'samsung',
 'intel',
 'c',
 'поиск',
 'инвестиции',
 'обучение',
 'node.js',
 'youtube',
 'оптимизация',
 'веб 2.0',
 'вконтакте',
 'flash',
 'ссылки',
 'windows phone',
 'дайджест',
 'веб-разработка',
 'алгоритмы',
 'музыка',
 'к

In [36]:
tags_to_save = set([
 'javascript',
 'linux',
 'php',
 'java',
 'python',
 'программирование',
 'разработка',
 '.net',
 'c++',
 'c#',
 'ubuntu',
 'ruby',
 'web-разработка',
 'api',
 'mysql',
 'c',
 'node.js',
 'веб-разработка',
 'алгоритмы',
 'виртуализация',
 'qt',
 'интерфейсы',
 'sql','машинное обучение', 'ml', 'dl', 'machine learning','docker'])

In [37]:
mask = df.tags.apply(lambda x: bool(list(set(x) & tags_to_save)))

In [45]:
res = df[mask]

In [46]:
res["text"] = res.text.apply(str)
res["text_len"] = res.text.apply(len)
res.text_len.mean()

C:\Users\ilove\AppData\Local\Temp\ipykernel_14728\3953190809.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res["text"] = res.text.apply(str)
C:\Users\ilove\AppData\Local\Temp\ipykernel_14728\3953190809.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res["text_len"] = res.text.apply(len)


8207.561475581639

In [69]:
res.text

0        Это первая запись в блог, чтобы проверить рабо...
1        Сегодня ночью наш Самый Главный по машинному к...
2        Как я понимаю, этот блог посвящен linux. Не зн...
3        В ЖЖ и, частично, на Linux.org.ru идёт активна...
4        Драйвера под Linux для клавиатуры Оптимус от с...
                               ...                        
23850    \nЗдрасте!\r\nВ последнее время все чаще задум...
23851    Современные фреймворки для разработки веб-прил...
23852    \n\r\nНесмотря на новогодние каникулы, в запад...
23853    Команда Rust рада сообщить о новой версии Rust...
23854    \r\nСейчас я изучаю отчёт очередной проверки п...
Name: text, Length: 23855, dtype: object

In [47]:
res.reset_index(drop=True, inplace=True)

In [48]:
res.to_csv("data/habr/clean_habr.csv")

In [52]:
res.text[5]

'Вчера стало известно, что Ганс Рейзер, разработчик файловой системы ReiserFS, арестован по подозрению в убийстве своей жены Нины Рейзер. \n\r\nReiserFS преследуют сплошные неудачи. Несмотря на все усилия разработчиков эту ФС так и не включили в ядро, в августе месяце команда была озабочена поиском работы, а совсем недавно Suse Labs отказались  от использования ReiserFS в качестве основной файловой системы для своей ОС. Теперь вот ещё и арест руководителя.\n\r\nВ связи с этими событиями дальнейшие перспективы развития ReiserFS выглядят туманными.'

In [54]:
ruABC = "ёйцукенгшщзхъфывапролджэячсмитьбюЁЙЦУКЕНГШЩЗХЪФЫВАПРОЛДЖЭЯЧСМИТЬБЮ"
enABC = "qwertyuiopasdfghjklzxcvbnmQWERTYUIOPASDFGHJKLZXCVBNM"
nums =  r'1234567890,.-)([]:@%№$" '
acceptable_chars = ruABC + enABC + nums

In [71]:
def convert_case(match_obj):
        if match_obj.group(1) is not None:
            return match_obj.group(1)

class PDFParser:
    
    SPECIAL_CHARS = "#^&*+_=<✓α𝑎>/\≡≡Σ∑∈●}{≤≥�åðÿæπ"
    NUMBERS = ("1","2", "3","4" ,"5" ,"6" ,"7" ,"8" ,"9")
    ACCEPTABLE_CHARS = acceptable_chars
    REPLACEMENT_DICT = {
        "»": r'"',
        "«": r'"',
        "”": r'"',
        "“": r'"',
        "—": r'-',
        "–": r'-'
    }
    
    def __init__(self):
        return
    
    @staticmethod
    def convert_case(match_obj):
        if match_obj.group(1) is not None:
            return match_obj.group(1)
    
    @staticmethod
    def delete_repeating_whitespaces(sent):
        return re.sub(' +', ' ', sent)
    
    @staticmethod
    def delete_unicode(sent):
        sent = re.sub('\xad', ' ', sent)
        return sent.encode("utf-8", "ignore").decode()

    @staticmethod
    def replace_hyphenation(sent):
        return re.sub("(\S)- ", convert_case, sent)
    
    
    def replace_chars(self, sent):
        for key in self.REPLACEMENT_DICT:
            sent = sent.replace(key, self.REPLACEMENT_DICT[key])
        return sent
    
    
    def mark_blocks(self):
        for block in self.blocks:
            if 84 < block['bbox'][0] < 86:
                block['type'] = "text"
            if block['lines'][0]['spans'][0]['font'] == 'CMUSerif-Bold':
                block['type'] = 'title'
            
    def blocks_to_text(self):
        textlines = [] 
        for block in self.blocks:
            block_textlines = []
            for line in block['lines']:
                for span in line['spans']:
                    block_textlines.append(span['text'])
            if not block_textlines[0].startswith("["):
                textlines += block_textlines
        return " ".join(textlines)
    
    def text_to_sents(self, text):
        sents = [sent.text for sent in list(sentenize(text))]        
        
        sents = list(filter(lambda x: not any (c in self.SPECIAL_CHARS for c in x), sents))
        sents = list(filter(lambda x: not x.startswith(self.NUMBERS), sents))
        sents = list(filter(lambda x: not "https:" in x, sents))
        sents = list(filter(lambda x: not re.search(r"[1-9]\.", x), sents))
        
        sents = [self.replace_chars(sent) for sent in sents] 
        sents = [self.delete_repeating_whitespaces(sent) for sent in sents]
        sents = [self.replace_hyphenation(sent) for sent in sents]
        sents = [self.delete_unicode(sent) for sent in sents]
        
        sents = list(filter(lambda x: all (c in self.ACCEPTABLE_CHARS for c in x), sents))
        sents = list(filter(lambda x: len(x) > 21, sents))
        sents = list(filter(lambda x: len(x) < 512, sents))
        
        return sents 
    
    def get_sentences(self, doc_path):
        try:
            doc = fitz.open(doc_path)
        except:
            return []
        
        self.blocks = []
        
        for page in doc:
            self.blocks += page.get_text("dict", flags=16)['blocks']

        self.mark_blocks()
        self.blocks = list(filter(lambda x: x['type'] == 'text', self.blocks))
        
        return self.text_to_sents(self.blocks_to_text())

In [72]:
parser = PDFParser()

sents = []
for text in tqdm(res.text):
    sents += parser.text_to_sents(text)

100%|███████████████████████████████████████████████████████████████████████████| 23855/23855 [02:01<00:00, 196.24it/s]


In [73]:
len(sents)

772987

In [76]:
for sent in sents[100000:110000]:
    print(sent)

По большому счету, Logback на сегодняшний день - вершина эволюции.
Помимо Logback появилось уже с десяток новых logging-библиотек, но с большой вероятностью ни одна из них не выживет.
JUL - тихо умирающий стандарт.
SLF4J - очень популярен в библиотеках.
Я уже говорил, что Open Source сообщество имеет тенденцию стекаться к "центрам тяжести".
Сейчас таким центром тяжести выступает скорее SLF4J в силу "универсальности".
Относительная популярность SLF4J в какой-то степени гарантирует от появления новых оберток.
Число проектов, использующих SLF4J, уже является достаточным для накопления "критической массы".
У Logback (того же автора, заметьте) такой критической массы нет.
Думаю, если Logback усмирит свою гордыню и двинется в Apache, его позиции сильно улучшатся.
То есть из двух вариантов "доработать существующее" и "сделать свое" всегда выбирался второй.
Поэтому ни один из упомянутых проектов не выбился в безусловные лидеры (в те самые стандарты "де-факто").
Вместо этого разработчики были в

Имена методов можно изменить, указав это явно.
Модули все еще включаются в цепочку наследования странным образом.
(Эта проблема так и не решена, удачи.
Потому что более короткое название метода означает, что он предпочтительнее к использованию.
Nakada [вероятно, Nobuyoshi Nakada, один из разработчиков ядра Ruby] уже начал разработку типажей, необходимых для реализации метода mix, в момент анонсирования этой функции на RubyKaigi [японская Ruby-конференция], так что патч уже готов.
Не предусмотрено никаких способов разрешения конфликтов среди переменных-членов класса.
Так как переменные-члены дочерних классов добавляются незаметно для родительского, при реализации класса, который смешивается с модулем, придется учитывать нежелательные имена для переменных-членов и избегать их.
Этой проблеме придется уделить больше внимания после введения смешивания.
Предложение заключается во введении метода prepend.
С этого вполне можно начать.
(На самом деле, true.)
Классы в Ruby являются открытыми.
Эт

Одно из его применений заключается в том, что он распознаёт в цепочке поступающих символов те или иные конструкции: слова, составные знаки препинания, функции, структуры, целые классы с их методами и полями.
Так действуют спеллчекеры, анализаторы исходного кода, компиляторы, инструменты для подсветки синтаксиса, так действует ваш компьютер, и прочее, и прочее.
Применимость у КА огромная.
Например, своим студентам я задавал сделать генерацию сказки с помощью недетерминированных КА, - и такое возможно.
Здесь применяется КА на стратегиях ([1]), написанный Сергеем Сацким ([1]) аж в 2003 году.
Конечный автомат Сацкого ([h]) реализован на паттерне "Стратегия".
(Strategy, [1], [2], [3]).
В качестве стратегий приходят типы SState, SEvent (событие, состояние), SFunctor (функтор перехода от одного состояния к другому) и SUnknownEventStrategy (поведение КА, если событие не распознано).
Пусть где-то во внешней среде мы перебираем текст посимвольно.
Мы берём очередной символ и передаем его в функци

Здесь и здесь можно найти более подробную информацию.
Т.к. любая ошибка генерирует исключение, такой код может сделать многие ошибки программирования похожими на ошибки времени исполнения, и затруднят отладку программы.
Поскольку "except" отлавливает все исключения, включая "SystemExit", "KeyboardInterrupt", и "GeneratorExit" (которые по сути не являются ошибками и не должны отлавливаться пользовательским кодом), использование голого "except" в любом случае плохая идея.
В ситуациях, когда нам нужно все-таки покрыть все возможные исключительные ситуации, мы можем использовать базовый класс для всех исключений - "Exception".
Есть ряд случаев, когда явные итераторы будут полезны.
Например, когда мы что-то обрабатываем, останавливаемся, делаем что-либо еще, затем возвращаемся назад и продолжаем.
Итератор в цикле проходит массив, выходит из цикла по break на индексе 5 (значение внутри равно 4).
Затем мы совершаем некие действия (выводим текст в консоль), и после этого продолжаем перебор ите

Мой клиент - это команда .NET разработчиков, а это значит, что двигаться можно в двух основных направлениях: разработка с использованием стандартных веб-технологий или Silverlight.
Когда необходимо рекомендовать одну технологию в противовес другой, то в идеале выбор должен быть основан на чем-то большем, чем на нескольких мнениях.
Мы создали список технологий-кандидатов.
После чего мы создали перечень критериев, связанных в категории.
Всем критериям был назначен некоторый вес, и мы оценили каждый из них для всех технологий-кандидатов.
В этой статье, я хотел бы пройтись только по категориям критериев и обсудить наши выводы.
Я поделюсь детальной таблицей с данными, для того чтобы вы смогли исследовать результаты самостоятельно.
В зависимости от ваших потребностей или вашего мнения, вы можете изменить вес и оценку, и посмотреть, как это повлияет на результат.
Я удалил несколько критериев, которые были специфичны для моего клиента, но это не вызвало огромного влияния на результат.
Для этой

Предварительно, выполняется предыдущий скрипт, разбирающий хард-линки.
Как видите, скрипт простейший и при желании его можно переписать под реакцию на любое другое изменение.
Можно повесить его cron и он будет сам следить за процессом, собирая по мере необходимости образы.
Обратная сторона - получаемый функционал: один раз собрав образ, вы получите возможность ввести в строй новый сервер за считанные минуты, а каждый собранный образ может служить точкой отката в случае неудачного обновления.
Разумеется здесь описаны только общие принципы и манипуляции, необходимые для получения базовой системы, запакованной в initrd с возможностью загрузки по сети.
Надеюсь, что это кому-нибудь будет пригодится.
Спасибо за внимание. :)
В эту субботу состоится очередной донецкий кофе-и-код.
Впервые у нас будет англоговорящий докладчик, Keith Gaddis, который является активным участником встреч рубистов в Остине, столице Техаса, который он решил покинуть на время гиковской вакханалии на ежегодном SXSW.
Kei

В этой статье речь пойдет о технологии JBFD, что означает Java BrainFuck Decompiler.
Технология еще достаточно молодая (от силы 3 часа), так что не судите строго.
Идея создания декомпилятора возникла не случайно.
Всему виной большое количество статей по BrainFuck в Интернете вообще и на Хабре в частности.
Интерпретаторов этого замечательного языка существует огромное множество, но вот средств для отладки BF кода мне удалось найти крайне мало.
Все это и много другое становится возможным с JBFD.
Для нетерпеливых, ссылка на скачивание исходного кода декомпиляторя находится здесь - JBFD.java.
Может кому-то будет полезно для валидации кода и выкидывания из исходников переводов строк и комментариев.
Теперь файл можно компилировать, дебажить, покрывать тестами, обфусцировать, в общем делать с ним все, что позволяет вам ваше воображение.
Декомпилятор тестировался пока что только на самых простых программах типа echo, hello world и т.д.
Код декомпилятора писался на скорую руку и будет дорабатыв

Для MyISAM разница между импортом и экспортом минимальна.
Эта "статья" вообще говоря в черновом варианте возникла несколько лет назад.
Сейчас сюда добавлен только сфинкс и заново проведены тесты.
А возникла она в результате спора на каком-то форуме, по поводу перспективности utf и что мол другие кодировки через год умрут.
Локаль в случае установки php как модуля едина на весь сервер со всеми вытекающими, и даже при правильной локали использовать обычные функции для работы со строками нельзя, надо использовать их аналоги поддерживающие эту работу.
P.S.: Тем кто любит сжимать траффик прокси-серверами на заметку - хтмл файл в утф8 на 5-20% больше, даже в gzip-е
Продолжаем тему операторов, на этот раз вас ждет рассказ об операторе запятая.
Последняя в списке значимых операторов, мало документированная, но очень полезная.
Она не так распространена, но она мне очень нравится.
Она проста, элегантна, и с ней лучше быть в хороших отношениях.
Оператор запятая выполняет оба операнда (слева направ

История будет и про сервера, и про Windows и Mac OS, про админские будни и праздники, и даже (немного) про Java.
Сперва немного о банальном.
Сервера в нашем мире чаще всего стоят в дата-центрах, доступ куда на "посмотреть" если и не сильно ограничен всяческими пропускными системами, то хотя бы неудобен из-за того, что датацентр обычно как назло на другом конце города.
Не наездишься на каждый сбой.
Но, цивилизация изобрела достаточное количество способов как-то достучаться до неблизких серверов.
А у нас же голосование идет, помогите, ну...".
Ну, собственно, и так далее.
Многие, кто держит сервера на площадках, и не в такие еще ситуации попадали.
Но нет, нас все же спасет прогресс: есть и более "идеологически верные" способы удаленного управления, полностью исключающие человеческий фактор.
Когда-то я преспокойно соединял сервера COM-портами "крест-накрест" и они замечательно давали управлять друг другом.
BIOS мог перенаправлять вывод в консоль, ОС сама после загрузки - тоже, а так как ве

- Тормознутый домашний сайт.
- Нестабильное время отклика.
Что-ж из всего вышесказаного, каждый сможет сделать выводы для себя сам.
Я для себя и своих задач выберу gevent.
UPD. немного ошибся со скриптом на gevent.
На него было меньше нагрузки.
Продажи смартфонов и планшетов растут с каждым днем, и это уже говорит о необходимости повышенного внимания к интерфейсам для мобильных устройств.
По имеющимся прогнозам, в течение 2011 года объем продаж планшетов существенно возрастет, а объем продаж смартфонов существенно превысит объем продаж телефонов традиционного образца.
Как показывает практика, пользователи предпочитают приложения для веб-навигации, уже установленные на телефоне, специальному программному обеспечению, которое нужно специально покупать или скачивать.
В первую очередь - то, что в настоящее время перед веб-разработчиками и дизайнерами стоит задача учета особенностей мобильных устройств.
Но при создании сайтов для мобильных устройств нужно двигаться еще дальше.
Семантическая

In [78]:
df = pd.DataFrame({"text": sents})
df.to_csv("data/sents/habr.csv", encoding='utf-8')